# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull.

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it.

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link.

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell.

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.*


# Importing Libraries

In [124]:
import os
import datetime
import re
import shutil

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random


---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways.

In [125]:
artists = {
    "coldplay": [
        "https://www.azlyrics.com/lyrics/coldplay/yellow.html",
        "https://www.azlyrics.com/lyrics/coldplay/fixyou.html",
        "https://www.azlyrics.com/lyrics/coldplay/clocks.html",
        "https://www.azlyrics.com/lyrics/coldplay/thescientist.html",
        "https://www.azlyrics.com/lyrics/coldplay/viva lavida.html",
        "https://www.azlyrics.com/lyrics/coldplay/shiver.html",
        "https://www.azlyrics.com/lyrics/coldplay/sparks.html",
        "https://www.azlyrics.com/lyrics/coldplay/trouble.html",
        "https://www.azlyrics.com/lyrics/coldplay/inmyplace.html",
        "https://www.azlyrics.com/lyrics/coldplay/loversinJapan.html",
        "https://www.azlyrics.com/lyrics/coldplay/42.html",
        "https://www.azlyrics.com/lyrics/coldplay/astronaut.html",
        "https://www.azlyrics.com/lyrics/coldplay/charliebrown.html",
        "https://www.azlyrics.com/lyrics/coldplay/magic.html",
        "https://www.azlyrics.com/lyrics/coldplay/skyfullofstars.html",
        "https://www.azlyrics.com/lyrics/coldplay/adventureofalifetime.html",
        "https://www.azlyrics.com/lyrics/coldplay/everyteardropisawaterfall.html",
        "https://www.azlyrics.com/lyrics/coldplay/hymnfortheweekend.html",
        "https://www.azlyrics.com/lyrics/coldplay/somethingjustlikethis.html",
        "https://www.azlyrics.com/lyrics/coldplay/championof the world.html"
    ],
    "strokes": [
        "https://www.azlyrics.com/lyrics/strokes/youonlyliveonce.html",
        "https://www.azlyrics.com/lyrics/strokes/lastnite.html",
        "https://www.azlyrics.com/lyrics/strokes/someday.html",
        "https://www.azlyrics.com/lyrics/strokes/hardtoexplain.html",
        "https://www.azlyrics.com/lyrics/strokes/reptilia.html",
        "https://www.azlyrics.com/lyrics/strokes/1251.html",
        "https://www.azlyrics.com/lyrics/strokes/undercontrol.html",
        "https://www.azlyrics.com/lyrics/strokes/juicebox.html",
        "https://www.azlyrics.com/lyrics/strokes/redlight.html",
        "https://www.azlyrics.com/lyrics/strokes/thesametimereveryyear.html",
        "https://www.azlyrics.com/lyrics/strokes/metabuilding.html",
        "https://www.azlyrics.com/lyrics/strokes/ontheradio.html",
        "https://www.azlyrics.com/lyrics/strokes/ode2.html",
        "https://www.azlyrics.com/lyrics/strokes/theendhasnoguns.html",
        "https://www.azlyrics.com/lyrics/strokes/eternityforaday.html",
        "https://www.azlyrics.com/lyrics/strokes/threatofjoy.html",
        "https://www.azlyrics.com/lyrics/strokes/dragqueen.html",
        "https://www.azlyrics.com/lyrics/strokes/tryingyourluck.html",
        "https://www.azlyrics.com/lyrics/strokes/visionsofdarkness.html",
        "https://www.azlyrics.com/lyrics/strokes/askmeanything.html"
    ]
}

# we'll use this dictionary to hold both the artist name and the link on AZlyrics

In [126]:
def generate_filename_from_link(link):
    if not link:
        return None
    name = link.replace("https", "").replace("http", "")
    name = name.replace(".html", "").replace("/lyrics/", "")
    name = name.replace("://", "").replace(".", "_").replace("/", "_")
    return name + ".txt"

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).)

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again.

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages.

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know?

A: <!-- Delete this comment and put your answer here. -->


In [127]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, links in artists.items():
    print(f"Using {len(links)} direct links for {artist}")
    lyrics_pages[artist] = links


Using 20 direct links for coldplay
Using 20 direct links for strokes


Let's make sure we have enough lyrics pages to scrape.

In [128]:
for artist, lp in lyrics_pages.items():
    assert(len(set(lp)) >= 20)


In [129]:
# Let's see how long it's going to take to pull these lyrics
# if we're waiting `5 + 10*random.random()` seconds
for artist, links in lyrics_pages.items() :
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For coldplay we have 20.
The full pull will take for this artist will take 0.06 hours.
For strokes we have 20.
The full pull will take for this artist will take 0.06 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part.

1. Create an empty folder in our repo called "lyrics".
1. Iterate over the artists in `lyrics_pages`.
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages.
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name.


In [130]:
def generate_filename_from_link(link) :

    if not link :
        return None

    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")

    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")

    # tack on .txt
    name = name + ".txt"

    return(name)


In [131]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") :
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [132]:
start = time.time()

for artist in lyrics_pages:
    os.makedirs(f"lyrics/{artist}", exist_ok=True)
    for link in lyrics_pages[artist]:
        try:
            r = requests.get(link)
            time.sleep(5 + 10 * random.random())
            soup = BeautifulSoup(r.text, "html.parser")
            title = soup.find("title").text
            lyrics_divs = soup.find_all("div", class_=None, id=None)
            lyrics_text = "\n".join([div.get_text(separator="\n") for div in lyrics_divs])
            filename = generate_filename_from_link(link)
            with open(f"lyrics/{artist}/{filename}", "w", encoding="utf-8") as f:
                f.write(title + "\n\n" + lyrics_text)
        except:
            print(f"Error scraping: {link}")

print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")


Total run time was 0.13 hours.


In [133]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.13 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [134]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text):
    return re.findall(r'\w+', text.lower())

## Checking Lyrics

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work.

In [135]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders :
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files :
        with open("lyrics/" + artist + "/" + f_name) as infile :
            artist_words.extend(words(infile.read()))


    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For coldplay we have 20 files.
For coldplay we have roughly 80 words, 4 are unique.
For strokes we have 20 files.
For strokes we have roughly 80 words, 4 are unique.
